<a href="https://colab.research.google.com/github/Krishnabhuvan/machinelearning-feature-engineering/blob/main/columntransformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

In [2]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

In [3]:
df = pd.read_csv("/content/covid_toy.csv")

In [4]:
df.head()

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


In [5]:
df.isnull().sum()

,0
age,0
gender,0
fever,10
cough,0
city,0
has_covid,0


In [6]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(df.drop(columns=['has_covid']),df['has_covid'],
                                                test_size=0.2)

In [7]:
X_train

,age,gender,fever,cough,city
14,51,Male,104.0,Mild,Bangalore
20,12,Male,98.0,Strong,Bangalore
5,84,Female,NaN,Mild,Bangalore
37,55,Male,100.0,Mild,Kolkata
80,14,Female,99.0,Mild,Mumbai
...,...,...,...,...,...
0,60,Male,103.0,Mild,Kolkata
8,19,Female,100.0,Strong,Bangalore
18,64,Female,98.0,Mild,Bangalore
44,20,Male,102.0,Strong,Delhi


In [9]:
si = SimpleImputer()
X_train_fever = si.fit_transform(X_train[['fever']])

# also the test data
X_test_fever = si.fit_transform(X_test[['fever']])

In [10]:
X_train_fever.shape

(80, 1)

In [11]:
oe = OrdinalEncoder(categories=[['Mild','Strong']])
X_train_cough = oe.fit_transform(X_train[['cough']])

# also the test data
X_test_cough = oe.fit_transform(X_test[['cough']])

In [14]:
ohe = OneHotEncoder(drop='first',sparse_output = False)
X_train_gender_city = ohe.fit_transform(X_train[['gender','city']])
X_test_gender_city = ohe.fit_transform(X_test[['gender','city']])

In [15]:
X_train_gender_city.shape

(80, 4)

In [16]:
X_train_age = X_train.drop(columns=['gender','fever','cough','city']).values


In [17]:
X_test_age = X_test.drop(columns=['gender','fever','cough','city']).values


In [20]:
X_train_transformed = np.concatenate((X_train_age,X_train_fever,X_train_gender_city,X_train_cough), axis=1)
X_test_transformed = np.concatenate((X_test_age,X_test_fever,X_test_gender_city,X_test_cough), axis=1)

In [21]:
X_train_transformed.shape


(80, 7)

In [22]:
from sklearn.compose import ColumnTransformer

In [24]:
transformer = ColumnTransformer(transformers=[
                                              ('tnf1',SimpleImputer(),['fever']),
                                             ('tnf2',OrdinalEncoder(categories=[['Mild','Strong']]),['cough']),
    ('tnf3',OneHotEncoder(sparse_output=False,drop='first'),['gender','city'])
],remainder='passthrough' )

In [25]:
transformer.fit_transform(X_train).shape

(80, 7)

In [26]:
transformer.transform(X_test).shape

(20, 7)